In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import streamlit as st

In [2]:
!dir

 El volumen de la unidad C es Windows
 El n�mero de serie del volumen es: A64F-3419

 Directorio de c:\Users\dllamas\Desktop\repo\olist-interactive-dashboad\notebooks

14/05/2025  12:28    <DIR>          .
13/05/2025  10:36    <DIR>          ..
14/05/2025  14:06           141.122 clean-data-notebook.ipynb
14/05/2025  12:28           854.041 ejercicios.ipynb
14/05/2025  14:06            81.443 use-data-notebook.ipynb
               3 archivos      1.076.606 bytes
               2 dirs  424.026.542.080 bytes libres


In [3]:
dir_prefix = '../resources/raw_data/olist_{}_dataset.csv'
df_customers_original = pd.read_csv(dir_prefix.format('customers'), encoding='utf-8')
df_customers = df_customers_original.copy()
df_customers.name = 'customers'

df_order_items_original = pd.read_csv(dir_prefix.format('order_items'), encoding='utf-8')
df_order_items = df_order_items_original.copy()
df_order_items.name = 'order_items'

df_order_payments_original = pd.read_csv(dir_prefix.format('order_payments'), encoding='utf-8')
df_order_payments = df_order_payments_original.copy()
df_order_payments.name = 'order_payments'

df_order_reviews_original = pd.read_csv(dir_prefix.format('order_reviews'), encoding='utf-8')
df_order_reviews = df_order_reviews_original.copy()
df_order_reviews.name = 'order_reviews'

df_orders_original = pd.read_csv(dir_prefix.format('orders'), encoding='utf-8')
df_orders = df_orders_original.copy()
df_orders.name = 'orders'

df_products_original = pd.read_csv(dir_prefix.format('products'), encoding='utf-8')
df_products = df_products_original.copy()
df_products.name = 'products'

df_sellers_original = pd.read_csv(dir_prefix.format('sellers'), encoding='utf-8')
df_sellers = df_sellers_original.copy()
df_sellers.name = 'sellers'

df_translations_original = pd.read_csv('../resources/raw_data/product_category_name_translation.csv', encoding='utf-8')
df_translations = df_translations_original.copy()
df_translations.name = 'translations'


In [4]:
all_df = [df_customers, df_order_items, df_order_payments, df_order_reviews, df_orders, df_products, df_sellers, df_translations]

In [5]:
for df in all_df:
    print(f"---- {df.name.upper()} ----", end='\n\n')
    print(df.info())
    print("\n---------------------------------------------\n")

---- CUSTOMERS ----

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None

---------------------------------------------

---- ORDER_ITEMS ----

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3

In [6]:
for df in all_df:
    print(f"---- {df.name.upper()} ----", end='\n\n')
    print(f"Parsing Object dtype to String dtype on {df.name.upper()}", end='\n\n')
    object_cols = df.select_dtypes(include='object').columns
    df[object_cols] = df[object_cols].astype(pd.StringDtype())
    print(df.info())
    print("\n---------------------------------------------\n")

---- CUSTOMERS ----

Parsing Object dtype to String dtype on CUSTOMERS

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  string
 1   customer_unique_id        99441 non-null  string
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  string
 4   customer_state            99441 non-null  string
dtypes: int64(1), string(4)
memory usage: 3.8 MB
None

---------------------------------------------

---- ORDER_ITEMS ----

Parsing Object dtype to String dtype on ORDER_ITEMS

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  string 
 

In [7]:
names_to_find = {'at', 'timestamp', 'date'}
names_to_exclude = {'state', 'status', 'category'}

for df in all_df:
    print(f"Buscando fechas en {df.name}: ")
    string_columns = df.select_dtypes(['object', 'string']).columns.values
    date_columns = [column for column in string_columns if 
        ((any(include in column.lower() for include in names_to_find)) 
            and not
         (any(exclude in column.lower() for exclude in names_to_exclude))
        ) ]

        
    if date_columns:
        print(f"Se encontraron columnas con fechas,\n{date_columns}")
        for column in date_columns:
            print(f"Parseando {column}")
            df[column] = pd.to_datetime(df[column])

    else:
        print("No se han encontrado columnas de Fechas")

    print()
    
        

Buscando fechas en customers: 
No se han encontrado columnas de Fechas

Buscando fechas en order_items: 
Se encontraron columnas con fechas,
['shipping_limit_date']
Parseando shipping_limit_date

Buscando fechas en order_payments: 
No se han encontrado columnas de Fechas

Buscando fechas en order_reviews: 
Se encontraron columnas con fechas,
['review_creation_date', 'review_answer_timestamp']
Parseando review_creation_date
Parseando review_answer_timestamp

Buscando fechas en orders: 
Se encontraron columnas con fechas,
['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
Parseando order_purchase_timestamp
Parseando order_approved_at
Parseando order_delivered_carrier_date
Parseando order_delivered_customer_date
Parseando order_estimated_delivery_date

Buscando fechas en products: 
No se han encontrado columnas de Fechas

Buscando fechas en sellers: 
No se han encontrado columnas de Fechas

Bu

In [8]:
[print(f'Nulos en {df.name} ->\n{df.isna().sum()}\n\n') for df in all_df]

Nulos en customers ->
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64


Nulos en order_items ->
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64


Nulos en order_payments ->
order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64


Nulos en order_reviews ->
review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64


Nulos en orders ->
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timesta

[None, None, None, None, None, None, None, None]

In [9]:
df_not_null_order_reviews = df_order_reviews.copy()
df_not_null_order_reviews.name = 'order_reviews_review_comments_no_null'
df_not_null_order_reviews.isna().sum()

review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

In [10]:
scores = df_not_null_order_reviews['review_score'].unique()

scores.sort()

scores

array([1, 2, 3, 4, 5])

In [11]:
for score in scores:
    print('-------------------------------------------------------------------------')
    df_by_score = df_not_null_order_reviews[df_not_null_order_reviews['review_score'] == score].copy()
    title_mode = df_by_score['review_comment_title'].mode()[0]
    msg_mode = df_by_score['review_comment_message'].mode()[0]
    print(f'Moda para el score -> {score}', f'Moda de Títutlo -> {title_mode}',  f'Moda de Message -> {msg_mode}',sep='\n', end='\n\n')
    print(f'Rellenando nulos para título y mensaje de review para el score -> {score}')
    print(f"Nulos antes de rellenar:\n {df_by_score.isna().sum()}", end='\n\n') 
    df_by_score.fillna({'review_comment_title': title_mode}, inplace=True)
    df_by_score.fillna({'review_comment_message': msg_mode}, inplace=True)
    print(f"Nulos después de rellenar:\n {df_by_score.isna().sum()}") 
    print('-------------------------------------------------------------------------')

    df_not_null_order_reviews[df_not_null_order_reviews['review_score'] == score] = df_by_score
    

-------------------------------------------------------------------------
Moda para el score -> 1
Moda de Títutlo -> Não recomendo
Moda de Message -> Não recebi o produto

Rellenando nulos para título y mensaje de review para el score -> 1
Nulos antes de rellenar:
 review_id                     0
order_id                      0
review_score                  0
review_comment_title       9551
review_comment_message     2679
review_creation_date          0
review_answer_timestamp       0
dtype: int64

Nulos después de rellenar:
 review_id                  0
order_id                   0
review_score               0
review_comment_title       0
review_comment_message     0
review_creation_date       0
review_answer_timestamp    0
dtype: int64
-------------------------------------------------------------------------
-------------------------------------------------------------------------
Moda para el score -> 2
Moda de Títutlo -> Bom
Moda de Message -> ok

Rellenando nulos para título y men

In [12]:
df_not_null_order_reviews.isna().sum()

review_id                  0
order_id                   0
review_score               0
review_comment_title       0
review_comment_message     0
review_creation_date       0
review_answer_timestamp    0
dtype: int64

In [13]:
df_orders.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [14]:
df_delivered_orders = df_orders.dropna()
df_delivered_orders.name = 'delivered_orders'
df_delivered_orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15


In [15]:
df_not_approved_orders = df_orders[df_orders['order_approved_at'].isna()].copy()
df_not_approved_orders.name = 'not_approved_orders'
df_not_approved_orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
1130,00b1cb0320190ca0daa2c88b35206009,3532ba38a3fd242259a514ac2b6ae6b6,canceled,2018-08-28 15:26:39,NaT,NaT,NaT,2018-09-12
1801,ed3efbd3a87bea76c2812c66a0b32219,191984a8ba4cbb2145acb4fe35b69664,canceled,2018-09-20 13:54:16,NaT,NaT,NaT,2018-10-17
1868,df8282afe61008dc26c6c31011474d02,aa797b187b5466bc6925aaaa4bb3bed1,canceled,2017-03-04 12:14:30,NaT,NaT,NaT,2017-04-10
2029,8d4c637f1accf7a88a4555f02741e606,b1dd715db389a2077f43174e7a675d07,canceled,2018-08-29 16:27:49,NaT,NaT,NaT,2018-09-13
2161,7a9d4c7f9b068337875b95465330f2fc,7f71ae48074c0cfec9195f88fcbfac55,canceled,2017-05-01 16:12:39,NaT,NaT,NaT,2017-05-30
...,...,...,...,...,...,...,...,...
97696,5a00b4d35edffc56b825c3646a99ba9d,6a3bdf004ca96338fb5fad1b8d93c2e6,canceled,2017-07-02 15:38:46,NaT,NaT,NaT,2017-07-25
98415,227c804e2a44760671a6a5697ea549e4,62e7477e75e542243ee62a0ba73f410f,canceled,2017-09-28 15:02:56,NaT,NaT,NaT,2017-10-16
98909,e49e7ce1471b4693482d40c2bd3ad196,e4e7ab3f449aeb401f0216f86c2104db,canceled,2018-08-07 11:16:28,NaT,NaT,NaT,2018-08-10
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,NaT,NaT,NaT,2018-10-01


In [16]:
df_not_picked_up_orders = df_orders[df_orders['order_delivered_carrier_date'].isna()].copy()
df_not_picked_up_orders.name = 'not_picked_up_orders'
df_not_picked_up_orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaT,NaT,2017-05-09
103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,NaT,NaT,2018-08-21
128,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,NaT,NaT,2017-10-03
266,8e24261a7e58791d10cb1bf9da94df5c,64a254d30eed42cd0e6c36dddb88adf0,unavailable,2017-11-16 15:09:28,2017-11-16 15:26:57,NaT,NaT,2017-12-05
324,d3c8851a6651eeff2f73b0e011ac45d0,957f8e082185574de25992dc659ebbc0,processing,2016-10-05 22:44:13,2016-10-06 15:51:05,NaT,NaT,2016-12-09
...,...,...,...,...,...,...,...,...
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,NaT,NaT,NaT,2018-10-01
99313,e9e64a17afa9653aacf2616d94c005b8,b4cd0522e632e481f8eaf766a2646e86,processing,2018-01-05 23:07:24,2018-01-09 07:18:05,NaT,NaT,2018-02-06
99347,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,2018-09-06 18:45:47,NaT,NaT,NaT,2018-09-27
99348,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,2017-08-28 15:44:47,NaT,NaT,2017-09-15


In [17]:
df_not_delivered_orders = df_orders[df_orders['order_delivered_customer_date'].isna()].copy()
df_not_delivered_orders.name = 'not_delivered_orders'
df_not_delivered_orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaT,NaT,2017-05-09
44,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,2018-06-04 16:44:48,2018-06-05 04:31:18,2018-06-05 14:32:00,NaT,2018-06-28
103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,NaT,NaT,2018-08-21
128,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,NaT,NaT,2017-10-03
154,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,2018-01-10 11:33:07,2018-01-11 02:32:30,2018-01-11 19:39:23,NaT,2018-02-07
...,...,...,...,...,...,...,...,...
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,NaT,NaT,NaT,2018-10-01
99313,e9e64a17afa9653aacf2616d94c005b8,b4cd0522e632e481f8eaf766a2646e86,processing,2018-01-05 23:07:24,2018-01-09 07:18:05,NaT,NaT,2018-02-06
99347,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,2018-09-06 18:45:47,NaT,NaT,NaT,2018-09-27
99348,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,2017-08-28 15:44:47,NaT,NaT,2017-09-15


In [18]:
[print(f'Nulos en {df.name} ->\n{df.isna().sum()}\n\n') for df in all_df]

Nulos en customers ->
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64


Nulos en order_items ->
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64


Nulos en order_payments ->
order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64


Nulos en order_reviews ->
review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64


Nulos en orders ->
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timesta

[None, None, None, None, None, None, None, None]

In [19]:
all_cleaned_df = [
df_customers,
df_orders,
df_delivered_orders,
df_not_approved_orders,
df_not_delivered_orders,
df_not_picked_up_orders,
df_order_items,
df_order_payments,
df_not_null_order_reviews,
df_products,
df_sellers,
df_translations,
]

In [20]:
for df in all_cleaned_df:
    print(f"Saving {df.name.lower()} as csv")
    df.to_csv(f'../resources/clean_data/clean_{df.name.lower()}.csv', encoding='utf-8', sep=',', index=False)

Saving customers as csv


Saving orders as csv
Saving delivered_orders as csv
Saving not_approved_orders as csv
Saving not_delivered_orders as csv
Saving not_picked_up_orders as csv
Saving order_items as csv
Saving order_payments as csv
Saving order_reviews_review_comments_no_null as csv
Saving products as csv
Saving sellers as csv
Saving translations as csv


In [21]:
score_map = {
'bad_review': [1,2],
'normal_review': 3,
'good_review': [4,5]
}